In [132]:
import nltk
from nltk import regexp_tokenize, word_tokenize
# nltk.download('punkt')
print("Text Processing With NLTK")

Text Processing With NLTK


In [133]:
corpus = {
    "simple_Text": "The brown fox jumps over the lazy dog",
    "punctuation": "Hello. Who am I speaking with? Dear Lord!",
    "contractions": "I'am am the one who knocks. Yes you're it, it's obvious",
    "numbers": "12.30$ 77.5% 499.99€ 0,77%",
    "compound_words": "guarda-chuva, nao-sei-mais",
    "abreviaturas": "U.S.A."
}
corpus_tokens = {
    'simple_Text': ['The', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog'],
    'punctuation': ['Hello', '.', 'Who', 'am', 'I', 'speaking', 'with', '?', 'Dear', 'Lord', '!'],
    'contractions': ["I'am", 'am', 'the', 'one', 'who', 'knocks', '.', 'Yes', "you're", 'it', ',', "it's", 'obvious'],
    'numbers': ['12.30$', '77.5%', '499.99€', '0,77%'],
    'compound_words': ['guarda-chuva', ',', 'nao-sei-mais'],
    'abreviaturas': ['U.S.A.'],
}

In [134]:
def print_tests(tests):
    passed = {name: test for name, test in tests.items() if test["passed"]}
    failed = {name: test for name, test in tests.items() if not test["passed"]}

    for key, test in passed.items():
        print(f"✅ {key}")
        print(f"      Output: {test['result']}")

    for key, test in failed.items():
        print(f"🚨 {key}")
        print(f"     Expected: {test['expected']}")
        print(f"     Got     : {test['result']}")


def test_tokenize(tokenizer):
    tests = {}
    for test, text in corpus.items():
        got = tokenizer(text)
        expected = corpus_tokens[test]
        tests[test] = {
            "result": got,
            "expected": expected,
            "passed": got == expected
        }
    print_tests(tests)

In [135]:
pattern = r'''(?x)
    (?:[a-zA-Z]\.)+    # Abreviaturas
    | (?:\w+)'(?:am|re|s|t)
    | (?:\w+(?:-\w+)+) # Compound words
    | \d+(?:[.,]\d+)?[$£%€]? # Numbers and currencies
    | \w+              # Normal words
    | [.,:-?!]         # Punctuation
    '''

test_tokenize(lambda text: regexp_tokenize(text, pattern))

✅ simple_Text
      Output: ['The', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']
✅ punctuation
      Output: ['Hello', '.', 'Who', 'am', 'I', 'speaking', 'with', '?', 'Dear', 'Lord', '!']
✅ contractions
      Output: ["I'am", 'am', 'the', 'one', 'who', 'knocks', '.', 'Yes', "you're", 'it', ',', "it's", 'obvious']
✅ numbers
      Output: ['12.30$', '77.5%', '499.99€', '0,77%']
✅ compound_words
      Output: ['guarda-chuva', ',', 'nao-sei-mais']
✅ abreviaturas
      Output: ['U.S.A.']


### Attention to Alternation

When making an disjunction  `|` in a regular expression, remember that the order in which they are defined matter.

So if you have *regexes* that are more **general** they should be further down.

In [136]:
pattern = r'''(?x)
    \w+
    | [.,?!]
    | \d+(?:[.,]\d+)?
'''
pattern_better = r'''(?x)
    [.,?!]
    |\d+(?:[.,]\d+)? 
    |\w+   # Most general in the end
'''
will_cause_problem = "That will be 69.99"

print(regexp_tokenize(will_cause_problem, pattern))
print(regexp_tokenize(will_cause_problem, pattern_better))

['That', 'will', 'be', '69', '.', '99']
['That', 'will', 'be', '69.99']


In [137]:
from nltk.tokenize import wordpunct_tokenize

test_tokenize(wordpunct_tokenize)

✅ simple_Text
      Output: ['The', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']
✅ punctuation
      Output: ['Hello', '.', 'Who', 'am', 'I', 'speaking', 'with', '?', 'Dear', 'Lord', '!']
🚨 contractions
     Expected: ["I'am", 'am', 'the', 'one', 'who', 'knocks', '.', 'Yes', "you're", 'it', ',', "it's", 'obvious']
     Got     : ['I', "'", 'am', 'am', 'the', 'one', 'who', 'knocks', '.', 'Yes', 'you', "'", 're', 'it', ',', 'it', "'", 's', 'obvious']
🚨 numbers
     Expected: ['12.30$', '77.5%', '499.99€', '0,77%']
     Got     : ['12', '.', '30', '$', '77', '.', '5', '%', '499', '.', '99', '€', '0', ',', '77', '%']
🚨 compound_words
     Expected: ['guarda-chuva', ',', 'nao-sei-mais']
     Got     : ['guarda', '-', 'chuva', ',', 'nao', '-', 'sei', '-', 'mais']
🚨 abreviaturas
     Expected: ['U.S.A.']
     Got     : ['U', '.', 'S', '.', 'A', '.']


### Punkt Sentence Tokenizer

This tokenizer divides a text into a list of sentences by using an **unsupervised algorithm** to build a model for *abbreviation words*, *collocations*, and *words that start sentences*.

It must be trained on a large collection of plaintext in the target language before it can be used.

The NLTK data package includes a pre-trained Punkt tokenizer for English.

`punkt` is a **sentence segementation** tokenizer

In [138]:
import nltk.data

punkt = nltk.data.load('tokenizers/punkt/english.pickle')

In [139]:
lines = '''
Punkt knows that the periods in Mr. Smith and Johann S. Bach
do not mark sentence boundaries.  and sometimes sentences
can start with non-capitalized words.  (How does it deal with 
this parenthesis?)  "It should be part of the
previous sentence." "(And the same with this one.)" ('And this one!')
"('(And (this)) '?)" [(and this. )]
'''

def test_sent_tokenizer(sent_tokenizer):
    print("\n------\n".join(sent_tokenizer(lines)))

In [140]:
test_sent_tokenizer(lambda lines : punkt.tokenize(lines))


Punkt knows that the periods in Mr. Smith and Johann S. Bach
do not mark sentence boundaries.
------
and sometimes sentences
can start with non-capitalized words.
------
(How does it deal with 
this parenthesis?)
------
"It should be part of the
previous sentence."
------
"(And the same with this one.)"
------
('And this one!')
------
"('(And (this)) '?)"
------
[(and this. )]


In [141]:
from nltk.tokenize import sent_tokenize
test_sent_tokenizer(sent_tokenize)


Punkt knows that the periods in Mr. Smith and Johann S. Bach
do not mark sentence boundaries.
------
and sometimes sentences
can start with non-capitalized words.
------
(How does it deal with 
this parenthesis?)
------
"It should be part of the
previous sentence."
------
"(And the same with this one.)"
------
('And this one!')
------
"('(And (this)) '?)"
------
[(and this. )]


## Sentence Segmentation on Long Texts


In [142]:
from urllib import request

url = "http://www.gutenberg.org/files/2554/2554-0.txt"
response = request.urlopen(url)
raw = response.read().decode('utf8')

In [143]:
print(f"Characters {len(raw)}")
print(f"Lines using python string.plit('\\n'):",len(raw.split('\n')))

Characters 1176812
Lines using python string.plit('\n'): 22444


In [144]:
lines = punkt.tokenize(raw)
tokens_by_lines = [word_tokenize(line) for line in lines]
vocabulary = {}
for line in tokens_by_lines:
    for first_token in line:
        vocabulary[first_token] = vocabulary.get(first_token, 0) + 1

print("Lines:", len(lines))
print("Tokens: ", sum([len(l) for l in tokens_by_lines]))
print("Vocabulary", len(vocabulary.keys()))

Lines: 12060
Tokens:  257058
Vocabulary 11516


In [145]:
# First Token in the second sentence
print(tokens_by_lines[1][0])

You


In [146]:
# Get N most frequent tokens
sorted_by_frequence = sorted(vocabulary.items(),key=lambda a : a[1],reverse=True)
print(sorted_by_frequence[:10])

[(',', 16177), ('.', 8908), ('the', 7447), ('and', 6279), ('to', 5280), ('a', 4469), ('I', 4397), ('’', 4039), ('“', 3980), ('”', 3929)]


### Using the counter Container from the builtin [collections](https://docs.python.org/3/library/collections.html#collections.Counter)

It is very handy and as we'll see it will spare us this previous work

In [147]:
from collections import Counter

frequency = Counter((token for line in tokens_by_lines for token in line))

In [148]:
frequency.most_common(10)

[(',', 16177),
 ('.', 8908),
 ('the', 7447),
 ('and', 6279),
 ('to', 5280),
 ('a', 4469),
 ('I', 4397),
 ('’', 4039),
 ('“', 3980),
 ('”', 3929)]

In [149]:
frequency.total()

257058

In [150]:
len(set(frequency))

11516

## Multi Word Expressions

As we know, this is a problem similar to **named entity recognition**, we can give a tokenizer a **dictionary of multi-word-expressions** 

In [151]:
from nltk import MWETokenizer

text = "Good muffins cost $3.88\nin New York."


multi_word_expressions = [('New','York'),('Real','Madrid')]

mwe = MWETokenizer(multi_word_expressions,separator="_")

tokens = word_tokenize(text)
mwes_tokens = mwe.tokenize(tokens)

# It now recognizes the Multi Word Expressions in the "knowledge base"
print(mwes_tokens)


['Good', 'muffins', 'cost', '$', '3.88', 'in', 'New_York', '.']


## Lemmatization and Stemming

Both *lemmatization* and *stemization* are techinques of normalizing and reducing the corpus.

However *lemmatization* is a more expensive process that aims to find the **root** of each word.

While *stemming* applies a set of transformations that aims to cut off word suffixes.

### Stemming

`nltk` includes the **Porter stemmer** that we've talked about

In [152]:
from nltk import PorterStemmer

porter_stemmer = PorterStemmer()

# The piece of text from the slides
sentence = '''The European Commission has funded a numerical study to analyze the purchase of a pipe organ with no noise
for Europe's organization. Numerous donations have followed the analysis after a noisy debate.'''

In [153]:
tokens = word_tokenize(sentence)
def show_statistics(tokens):
    cnt = Counter(tokens)
    print(f"Number of Tokens:",cnt.total())
    print(f"Vocabulary:",len(set(cnt)))
show_statistics(tokens)

Number of Tokens: 35
Vocabulary: 31


In [154]:
# Now applying the Porter Stemmer to Normalize the text

stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
cnt = Counter(stemmed_tokens)
show_statistics(stemmed_tokens)

Number of Tokens: 35
Vocabulary: 28


As we can see the dimension of the vocabulary was reduced, but lets checkout what happened to the sentence:

In [155]:
print("Original: "," ".join(tokens))
print("Stemmed: "," ".join(stemmed_tokens))

Original:  The European Commission has funded a numerical study to analyze the purchase of a pipe organ with no noise for Europe 's organization . Numerous donations have followed the analysis after a noisy debate .
Stemmed:  the european commiss ha fund a numer studi to analyz the purchas of a pipe organ with no nois for europ 's organ . numer donat have follow the analysi after a noisi debat .


In [156]:
mapping = {}
for first_token in tokens:
    stemmed = porter_stemmer.stem(first_token)
    mapping[stemmed] = mapping.get(stemmed,set())
    mapping[stemmed].add(first_token)

sorted(mapping.items(),key=lambda a: len(a[1]),reverse=True)[:5]

[('the', {'The', 'the'}),
 ('numer', {'Numerous', 'numerical'}),
 ('organ', {'organ', 'organization'}),
 ('european', {'European'}),
 ('commiss', {'Commission'})]

As we can see some words suffered from **overgeneralization**, like *organ* and *organization*

And other words suffered from **undergeneralization** like **european** and **europe** that dind't got the same stem.

### Lemmatization

`nltk` includes ways of finding the root of words

In [157]:
# WordNet lemmatizer
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
# Init the Wordnet Lemmatizer
lemmatizer = WordNetLemmatizer()

sentence = "Men and women love to study artificial intelligence while studying data science. Don't you? My feet and teeth are clean!"

[nltk_data] Downloading package wordnet to /home/martim/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [158]:
tokens = word_tokenize(sentence)
show_statistics(tokens)

Number of Tokens: 24
Vocabulary: 23


In [159]:
lemmatized_tokens = [lemmatizer.lemmatize(t) for t in tokens]
print(lemmatized_tokens)
show_statistics(lemmatized_tokens)

stemmed_tokens = [porter_stemmer.stem(t) for t in tokens]
show_statistics(stemmed_tokens)


import pandas as pd
def compare_lemma_stemmer(tokens):
    data = []
    for token in tokens:
        data.append([token,porter_stemmer.stem(token),lemmatizer.lemmatize(token)])
    return pd.DataFrame(data,columns=["Original","Stemmed","Lemmatized"])

compare_lemma_stemmer(word_tokenize("cats corpora mice")) 


['Men', 'and', 'woman', 'love', 'to', 'study', 'artificial', 'intelligence', 'while', 'studying', 'data', 'science', '.', 'Do', "n't", 'you', '?', 'My', 'foot', 'and', 'teeth', 'are', 'clean', '!']
Number of Tokens: 24
Vocabulary: 23
Number of Tokens: 24
Vocabulary: 22


,Original,Stemmed,Lemmatized
0,cats,cat,cat
1,corpora,corpora,corpus
2,mice,mice,mouse


To improve the performance of the `lemmatizer` it needs the **part of speech** of the token, by default it uses *noun*, so the lemmatizer only removed plurals

# spaCy


`spaCy` is a python library that provides several **language processing pipelines** that streamline and facilitate the process of language processing.

Once a `language pipeline` is loaded  it will return a `Language` that is an object that contains all the components and data needed to process text.

Those compoenents are:
- Binary Weights of a model for the **part-of-speech tagger**, the **dependency parser** and the **named entity recognizer** to predict the annotations in the text
- Lexical Entries in the vocabulary: words and their context independent attributes like shape and spelling
- Data files for lemmatization  rules and look up tables
- Word Vectors multidimensional meaning representations of the words that let you determine how similar words are

Between other

In [160]:
# pip install -U spacy
# python -m spacy download en_core_web_sm
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_sm")

### Processing With SpaCy

Now we just need to call `nlp(text)` and it will return an object of the type `Document` that is the processed text. However it is worth mentioning that the `Document` still holds all the information about the text and it is possible to reconstruct it from the `Document`

In [161]:
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

In [162]:
# Just for pretty printing
import pandas as pd

pd.DataFrame(([t.text,t.pos_,t.dep_] for t in doc),columns=['Text','POS','Dependency'])

,Text,POS,Dependency
0,Apple,PROPN,nsubj
1,is,AUX,aux
2,looking,VERB,ROOT
3,at,ADP,prep
4,buying,VERB,pcomp
5,U.K.,PROPN,dobj
6,startup,NOUN,dep
7,for,ADP,prep
8,$,SYM,quantmod
9,1,NUM,compound


As we can see the ease of use of `spaCy` as just calling the **language processing pipeline** was easy and gives us lots of information

### Spacy's Tokenizer

It is **language dependent** as different languages have differences in how they should be tokenized.*Like the lack of spaces*

![](https://spacy.io/images/tokenization.svg)

Here we show the flow of the tokenizer, it first splits by spaces then tries to see if each "word" matches an exception rule and should be further devided.

It also tries to split off infixes, like punctuation.

In [163]:
columns_func = [
    ['text', 'text'], ['lemma', 'lemma_'],
    ['pos', 'pos_'], ['tag', 'tag_'],
    ['dep', 'dep_'], ['shape', 'shape_'],
    ['alpha', 'is_alpha'], ['stop', 'is_stop']
]

data = []
for tok in doc:
    row = []
    for _, func in columns_func:
        row.append(getattr(tok,func))
    data.append(row)

pd.DataFrame(data,columns=[i[0] for i in columns_func])

,text,lemma,pos,tag,dep,shape,alpha,stop
0,Apple,Apple,PROPN,NNP,nsubj,Xxxxx,True,False
1,is,be,AUX,VBZ,aux,xx,True,True
2,looking,look,VERB,VBG,ROOT,xxxx,True,False
3,at,at,ADP,IN,prep,xx,True,True
4,buying,buy,VERB,VBG,pcomp,xxxx,True,False
5,U.K.,U.K.,PROPN,NNP,dobj,X.X.,False,False
6,startup,startup,NOUN,NN,dep,xxxx,True,False
7,for,for,ADP,IN,prep,xxx,True,True
8,$,$,SYM,$,quantmod,$,False,False
9,1,1,NUM,CD,compound,d,False,False


In [164]:
doc = nlp("I ate fish and chips, in the U.K.")
displacy.render(doc, style="dep")

In [165]:
spacy.explain("ADJ")

'adjective'

In [166]:
doc = nlp("Apple is looking at buying U.K. startup for $1 billion")

data = []
for ent in doc.ents:
    data.append([ent.text, ent.start_char, ent.end_char, ent.label_,spacy.explain(ent.label_)])

pd.DataFrame(data,columns=["Text","Start","End","Label","Description"])


,Text,Start,End,Label,Description
0,Apple,0,5,ORG,"Companies, agencies, institutions, etc."
1,U.K.,27,31,GPE,"Countries, cities, states"
2,$1 billion,44,54,MONEY,"Monetary values, including unit"


In [167]:
displacy.render(doc,style="ent")

## Word Vectors and similarity

**Similarity** is determined by comparing *word vectors* or *word embeddings*, these are multi dimensional representations of a word.

Spacy pipelines that end in `sm` don't ship with *word vectors*, to get accurate similarity results a pipeline that ends in `lg` is required.

It is important to note that word similarity depends on the application needs as the sentence `I like pasta` and `I like pizza` are both similar 
because they both talk about food preferences but if our application is about differences in food, pizza and pasta are very different.

`Doc` and `Span` vector values are the **average** of its constituints. Meaning that ordering between words is lost.

In [168]:
import spacy

# python3 -m spacy download en_core_web_md
nlp = spacy.load("en_core_web_md")  # make sure to use larger package!
doc1 = nlp("I like salty fries and hamburgers.")
doc2 = nlp("Fast food tastes very good.")

# Similarity of two documents
print(doc1, "<->", doc2, doc1.similarity(doc2))

I like salty fries and hamburgers. <-> Fast food tastes very good. 0.691649353055761


In [169]:
# Similarity of tokens and spans
french_fries = doc1[2:4]
burgers = doc1[5]
print(french_fries, "<->", burgers, french_fries.similarity(burgers))

salty fries <-> hamburgers 0.6938489675521851


In [170]:
good = nlp('sweet')
bad = nlp('salt')
print(good,"<->",bad,good.similarity(bad))

sweet <-> salt 0.3181479327298317


![Pipeline](https://spacy.io/images/pipeline.svg)

![Architecture](https://spacy.io/images/architecture.svg)

In [171]:
pd.DataFrame(word_tokenize("its water is so transparent that"),columns=["Tokens"])

,Tokens
0,its
1,water
2,is
3,so
4,transparent
5,that


In [172]:
from nltk import sent_tokenize, word_tokenize

S_START, S_END = "<s>", "</s>"


def tokenize_sent(sentence):
    return [S_START] + word_tokenize(sentence) + [S_END]


corpus = "I am Sam and I like ham green. Sam I am and I am not ham. I do not like green eggs and ham."
sents = sent_tokenize(corpus)
sents_toks = [tokenize_sent(s) for s in sents]

bigrams = {}
vocabulary = {}
for sentence in sents_toks:
    for i in range(len(sentence)):
        first_token = sentence[i]
        frequency = vocabulary.get(first_token, 0) + 1
        vocabulary[first_token] = frequency
        if i < len(sentence) - 1:
            bigram = (first_token, sentence[i+1])
            frequency = bigrams.get(bigram,0) + 1
            bigrams[bigram] = frequency

bigrams_probability = {}
for bigram,bigram_frequency in bigrams.items():
    w0, w1 =  bigram
    frequency_w0 = vocabulary[w0]
    bigram_probability = bigram_frequency / frequency_w0
    bigrams_probability[bigram] = bigram_probability

def create_conditional_ngram(ngram):
    n = len(ngram)
    sequence,last = ngram[:n-1],ngram[n-1]
    return f'P({last}|{" ".join(sequence)})'

data = [[create_conditional_ngram(b[0]),b[1]] for b in bigrams_probability.items()]    
pd.DataFrame(data,columns=["Bigram","Probability"]).head(5)

,Bigram,Probability
0,P(I|<s>),0.666667
1,P(am|I),0.600000
2,P(Sam|am),0.333333
3,P(and|Sam),0.500000
4,P(I|and),0.666667


In [173]:
# Going for a more tabular approach
table = {}
vocabulary = {}
for sentence in sents_toks:
    for index, first_token in enumerate(sentence):
        frequency = vocabulary.get(first_token, 0) + 1
        vocabulary[first_token] = frequency
        if index < (len(sentence) - 1):
            next_token = sentence[index + 1]
            start_by_token = table.get(first_token, {})
            bigram_frequency = start_by_token.get(next_token, 0) + 1
            start_by_token[next_token] = bigram_frequency
            table[first_token] = start_by_token

# Set frequency to 0 to bigrams that don't appear in the corpus
for _, start_by in table.items():
    for first_token in vocabulary:
        if not first_token in start_by:
            start_by[first_token] = 0
pd.DataFrame(table).transpose()

,I,Sam,<s>,am,and,like,ham,green,.,</s>,not,do,eggs
<s>,2,1,0,0,0,0,0,0,0,0,0,0,0
I,0,0,0,3,0,1,0,0,0,0,0,1,0
am,0,1,0,0,1,0,0,0,0,0,1,0,0
Sam,1,0,0,0,1,0,0,0,0,0,0,0,0
and,2,0,0,0,0,0,1,0,0,0,0,0,0
like,0,0,0,0,0,0,1,1,0,0,0,0,0
ham,0,0,0,0,0,0,0,1,2,0,0,0,0
green,0,0,0,0,0,0,0,0,1,0,0,0,1
.,0,0,0,0,0,0,0,0,0,3,0,0,0
not,0,0,0,0,0,1,1,0,0,0,0,0,0


In [174]:
bigrams_probability = {}
for first_token, second_tokens_frequency in table.items():
    bigrams = {}
    first_token_frequency = vocabulary[first_token]
    for second_token, bigram_frequency in second_tokens_frequency.items():
        bigram_probability = bigram_frequency / first_token_frequency
        bigrams[second_token] = bigram_probability
    bigrams_probability[first_token] = bigrams

pd.DataFrame(bigrams_probability).transpose()

,I,Sam,<s>,am,and,like,ham,green,.,</s>,not,do,eggs
<s>,0.666667,0.333333,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
I,0.000000,0.000000,0.0,0.6,0.000000,0.2,0.000000,0.000000,0.000000,0.0,0.000000,0.2,0.0
am,0.000000,0.333333,0.0,0.0,0.333333,0.0,0.000000,0.000000,0.000000,0.0,0.333333,0.0,0.0
Sam,0.500000,0.000000,0.0,0.0,0.500000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
and,0.666667,0.000000,0.0,0.0,0.000000,0.0,0.333333,0.000000,0.000000,0.0,0.000000,0.0,0.0
like,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.500000,0.500000,0.000000,0.0,0.000000,0.0,0.0
ham,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.333333,0.666667,0.0,0.000000,0.0,0.0
green,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.500000,0.0,0.000000,0.0,0.5
.,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,1.0,0.000000,0.0,0.0
not,0.000000,0.000000,0.0,0.0,0.000000,0.5,0.500000,0.000000,0.000000,0.0,0.000000,0.0,0.0


In [175]:
def sequence_probability(sequence):
    tokens = word_tokenize(sequence)
    probability = 1
    for i in range(len(tokens) -1):
        w0,w1 = tokens[i],tokens[i+1]
        bigram_probability = bigrams_probability[w0][w1]
        probability = probability * bigram_probability
    return probability

sequence_probability('I am Sam and I am not like green eggs')

0.0016666666666666661

### Linguistic Phenomena

Each language properties will define the bigram probabilities, there will be corpus specific quirks of our data, like baises and patterns. And the culture affects the corpus.

## Practical Issues

A trigram or a 4-gram, 5-gram can **capture longer distance dependencies**, given enough training data however n-gram models have difficult in capturing dependencies in sentences like:

> The **computer** which I had just put into the machine room on the fith floor **crashed**

Multiplying many values between 0 and 1 can get very small numbers that can lead to numerical underflow and loss of precision

To solve this issue we use the follwoing property:

$$log(a \times b) = log(a) + log(b)$$
$$\exp(log(a \times b)) = \exp(log(a) + log(b))$$
$$ a \times b = \exp(log(a) + log(b))$$
In our probabilities:
$$ p_1 \times p_2 = \exp(log(p_1)+log(p_2))$$

This way we can avoid doing the multiplications

> Multiplying is the same as adding in the log space!

## Sparsity and Unknown words

N-gram models will workd well if the **training and test corpus look similar**, this means that there are no n-grams in the test corpus that didn't exist in the training corpus.

If that situation occurs they will have a **probability of zero**.

**Zero probaility n-grams**
- Missing in the trianing corpus but present in the testing corpus
- This means we underestimate the probability, and it is impossible to calculate as we will be dividing by 0

**Out of vocabulary word (OOV)**
What happens if in the testing corpus appear words that aren't in the vocabulary? This will result in a problem of dividing by 0 and underestimating.

There are two approaches to solve this problem:

- **Open vocabulary**: we model unknown words in the **test set** by adding a pseudo-word `<UNK>`

- **Closed vocabulary**: we assume every possible word of interest is known in advance and *(word list)* and convert any *out of vocabulary* word  in the **training corpus** to `<UNK>`. In this way the model will learn how to deal with unknowns.

## Smoothing

What we've learn from words and ngrams that don't appear in the training set is that we want to *avoid assigning zero probability to unseen events*.

*Smoothing* techniques consist in **shaving off a bit of probability mass from some more frequent events and give it to unseen ones

In [176]:
corpus = ("denied the allegations. " * 5
          + "denied the speculation. " * 2
            + "denied the rummors. " * 1
            + "denied the report. " * 1)
test_corpus = "denied the offer. denied the loan."

doc = []
for sent in sent_tokenize(corpus):
  sent_toks = [S_START]
  for tok in word_tokenize(sent):
      sent_toks.append(tok)
  sent_toks.append(S_END)
  doc.append(sent_toks)

In [177]:
from collections import defaultdict

bigrams = defaultdict(lambda: defaultdict(int))
vocabulary = defaultdict(int)

for sentence in doc:
    for index, token in enumerate(sentence):
        vocabulary[token] += 1
        if index < (len(sentence) - 1):
            next_token = sentence[index+1]
            bigrams[token][next_token] += 1

# To instantiate the bigrams of 0 frequency
for token in bigrams.keys():
    for next_token in vocabulary.keys():
        bigrams[token][next_token]
pd.DataFrame(bigrams).transpose()

,denied,<s>,the,allegations,.,</s>,speculation,rummors,report
<s>,9,0,0,0,0,0,0,0,0
denied,0,0,9,0,0,0,0,0,0
the,0,0,0,5,0,0,2,1,1
allegations,0,0,0,0,5,0,0,0,0
.,0,0,0,0,0,9,0,0,0
speculation,0,0,0,0,2,0,0,0,0
rummors,0,0,0,0,1,0,0,0,0
report,0,0,0,0,1,0,0,0,0


As we are using **bi-grams** we know that:

$$P(w_1|w_0) = \frac{C(w_0w_1)}{C(w_0)}$$

In [178]:
bigrams_probability = {}

for start_token,next_tokens in bigrams.items():
    start_token_frequency = vocabulary[start_token]
    bigrams_probability[start_token] = {}
    for next_token,bigram_frequency in next_tokens.items():
        probability = bigram_frequency / start_token_frequency
        bigrams_probability[start_token][next_token] = probability
pd.DataFrame(bigrams_probability).transpose()

,denied,<s>,the,allegations,.,</s>,speculation,rummors,report
<s>,1.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000
denied,0.0,0.0,1.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000
the,0.0,0.0,0.0,0.555556,0.0,0.0,0.222222,0.111111,0.111111
allegations,0.0,0.0,0.0,0.000000,1.0,0.0,0.000000,0.000000,0.000000
.,0.0,0.0,0.0,0.000000,0.0,1.0,0.000000,0.000000,0.000000
speculation,0.0,0.0,0.0,0.000000,1.0,0.0,0.000000,0.000000,0.000000
rummors,0.0,0.0,0.0,0.000000,1.0,0.0,0.000000,0.000000,0.000000
report,0.0,0.0,0.0,0.000000,1.0,0.0,0.000000,0.000000,0.000000


As we can see there are a lot of zeros and sparsity in this table. So we will use a technique called **Laplace Smoothing**

### Laplace Smoothing 

Also called **add one smoothing**, it adds one to all counts:
- **Unigrams**
$$P(w_i) = \frac{c_i}{N} \rarr P_{Laplace}(w_i) = \frac{c_i+1}{N + V}$$

- **Bigrams**

$$P(w_n|w_{n-1}) = \frac{C(w_{n-1}w_n)}{w_{n-1}} \rarr P_{Laplace}(w_n |w_{n-1}) = \frac{C(w_{n-1}w_n) +1}{C(w_{n-1}) + V}$$

Wait why does  $C(w_{n-1})$ becomes $C(w_{n-1}) + V$?

Because we **add one to all counts**. 

In unigram we add 1 to all words and in the n-gram model we add one to all ngrams.

So the frequncy of $w_{n-1} = \Sigma_{w \in V}C(w_{n-1}w)$ 

Before this was equal to the frequency of $C(w_{n-1})$ but now it is equal to $C(w_{n-1}) + V$ 
        

In [179]:
V = len(vocabulary)
laplace_vocabulary = {token: freq + V for token, freq in vocabulary.items()}
laplace_bigrams = {}
for token, next_tokens in bigrams.items():
    laplace_bigrams[token] = {}
    for next_token, frequency in next_tokens.items():
        laplace_bigrams[token][next_token] = frequency + 1

pd.DataFrame(laplace_bigrams).transpose()

,denied,<s>,the,allegations,.,</s>,speculation,rummors,report
<s>,10,1,1,1,1,1,1,1,1
denied,1,1,10,1,1,1,1,1,1
the,1,1,1,6,1,1,3,2,2
allegations,1,1,1,1,6,1,1,1,1
.,1,1,1,1,1,10,1,1,1
speculation,1,1,1,1,3,1,1,1,1
rummors,1,1,1,1,2,1,1,1,1
report,1,1,1,1,2,1,1,1,1


In [180]:
laplace_probabilities = {}
for token, next_tokens in laplace_bigrams.items():
    laplace_probabilities[token] = {}
    token_frequncy = laplace_vocabulary[token]
    for next_token,bigram_frequency in next_tokens.items():
        probability = bigram_frequency / token_frequncy
        laplace_probabilities[token][next_token] = probability
pd.DataFrame(laplace_probabilities).transpose()

,denied,<s>,the,allegations,.,</s>,speculation,rummors,report
<s>,0.555556,0.055556,0.055556,0.055556,0.055556,0.055556,0.055556,0.055556,0.055556
denied,0.055556,0.055556,0.555556,0.055556,0.055556,0.055556,0.055556,0.055556,0.055556
the,0.055556,0.055556,0.055556,0.333333,0.055556,0.055556,0.166667,0.111111,0.111111
allegations,0.071429,0.071429,0.071429,0.071429,0.428571,0.071429,0.071429,0.071429,0.071429
.,0.055556,0.055556,0.055556,0.055556,0.055556,0.555556,0.055556,0.055556,0.055556
speculation,0.090909,0.090909,0.090909,0.090909,0.272727,0.090909,0.090909,0.090909,0.090909
rummors,0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000
report,0.100000,0.100000,0.100000,0.100000,0.200000,0.100000,0.100000,0.100000,0.100000


A we can see:

- Smoothing **solves** the problem of unseen sequences of words.

But smoothing cannot solve the problem of **out of vocabulary words**

**Add 1 smoothing** can be too blunt 
- But it is still used in NLP models for text classification
- It is useful when the number of zeros is not too large (???? why????)

### Other Smooting techniques

**Add k smoothing**: move a bit less of the probability mass, where $k \in ]0,1[$

$$P^*_{Add-k}(w_n|w_{n-1}) = \frac{C(w_{n-1}w_n) + k}{C(w_{n-1})+ kV}$$

**Backoff  and interpolation** estimate n-gram probabilities using (n-1)-gram
*probabilities (the base case will be unigram probabilities, what if it is an
*OOV* word)
- It will be an weighted sum where all weights summ up to 1 $\Sigma_{k=1}^{n} \lambda_k = 1$

$$\hat{P}(w_n|w_{n-2}w_{n-1}) = \lambda_1 P(w_n|w_{n-2}w_{n-1})\\
+ \lambda_2  P(w_n|w_{n-1}) \\
+ \lambda_3  P(w_n) \\
$$


## Evalutaing Language Models

To tell a good model from a bad one we need a way of evaluating them.

**Extrinsic evaluation** is a time consuming process that uses the newly created model embeded in some other task and tell if that task got better or worse comparing with some other model.


**Intrinsic evaluation** this is the more interest process where we:
- Hold out a **test set** of the corpus
- Which language model best predicts an unseen test set meaning which model **assigns an higher probability to each of the training set sentences*

Here we use a the concept of **prepexity** that is the normalize inverse of the probability therefore minimizing the perplexity is maximizing the probability.

## Perplexity

Perplexity is the inverse probability of the test set, normalized by the number of words.


$$PP(W) = P(w_1w_2...w_N)^{-\frac{1}{N}}$$
$$PP(W) = \sqrt[N]{\frac{1}{P(w_1w_2...w_N)}}$$
$$PP(W) = \sqrt[N]{\prod_{k=1}^{N}\frac{1}{P(w_k|w_1^{k-1})}}$$

In the **bigram model** and using using the markov assumption:

$$PP(W) = \sqrt[N]{\prod_{k=1}^N \frac{1}{P(w_k|w_{k-1})}}$$

### Perplexity and Information Theory

Perplexity can also be seen as a **weighted branching factor** of a language
- *weighing each possible next word by its probability*

In a sequence of random digits where $P(d) = 1/10$ the $PP = \left[\frac{1}{10}^N\right]^{\frac{-1}{N}} = \frac{1}{10}^{-1} = 10$    

If we had the case where 0 was more frequent in the training data set such that:

$P(0) = 0.91, P(d) = 0.01, d\in [1,9]$

`0 0 0 0 3 0 0 0 0 0` $PP = [0.91^9 \times 0.01]^{-\frac{1}{10}} = 1.73$

`0 1 2 3 4 5 6 7 8 9` $PP = [0.91 \times 0.01^9]^{-\frac{1}{10}} = 63.69$

The model would be *perplexed to see `0 1 2 3 4 5 6 7 8 9`  in the **testing data**

## Training Corpus

Longer context brings more **coherent** sentences but also less generalization meaning some there can be direct transcripts of the training corpus in generated text

The model strongly depends on its **training corpus**, the language, genre and dialect need to be chosen according to the task.

## Generating Text

**Generating random sentences** from different n-gram models:
1. Assign a slice of `[0...1]` to each possible next word proportional to its relative probability
1. Generate a random value in the same space and choose the word whose slice includes the generated value.
1. Repeat this process until the generated word is a special token like `</s>`.

![](./ngram_text_generation.png)

Were we can see that coherence increases with the N of the N-gram, because context increases.

## Web Scale N-grams

Efficiency
- Words stored as 64-bit hashes indexes
- Efficient data structures
- Bloom filters to approximate language models

Prunning:
- Only store n-grams with counts above a certain threshold
- Use entropy to prune less important ngrams

### Smoothing in Web-Scale N-Grams

**Stupid Backoff**
Main idea : Give up trying to make the language model a true probability distribution

There fore no "shaving off probabilities": 
- if an n-gram has 0 frequency then backoff to a lower order n-gram weighted by a fixed weight

This means: loose a bit of context by going to a lower order n-gram

$$S(w_i|w_{i-k+1}^{i-1}) =
\begin{cases} 
\frac{C(w_{i-k+1}^{i-1}w_i)}{C(w_{i-k+1})},  && C(w_{i-k+1}^{i-1}w_1) \gt 0 \\
\lambda S(w_i|w_{i-k+2}^{i-1}),               &&  \text{ otherwise}
\end{cases}


## Problems with N-gram Language Models

- **Sparsity problem** what if a particular sequence never appeared in the training corpus?
    - **smoothing** partially solves this problem

Markov models have limited contextual information

**Storage cost** need to store frequencies for all possible n-grams.

In summary these models are very sparse and they treat all words/ prefixes independently of each other

```
Students opened their ___
Pupils opened their ___
Scholars opened their ___
Students turned the pages of their ___
```
Could we **share information** across these semantically similar prefixes?

# Neural Language Models

Models based on **word embedding** / **word vectors** they can generalize over contexts of similar words and don't need smoothing, can handle much larger histories  and typically can have **higher predictive accuracy** than n-gram models

At the cost of much greater training times.